In [1]:
!pip install streamlit pyngrok PyJWT bcrypt --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 55.7 MB/s eta 0:00:00


In [2]:
%%writefile app.py
import streamlit as st
import sqlite3
import re
import jwt
import datetime
import bcrypt

SECRET_KEY = "policynav_secret_key"

# ================= CUSTOM CSS =================
def load_css():
    st.markdown("""
    <style>
    /* Main background with gradient */
    .stApp {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    }

    /* Custom container styling */
    .main-container {
        background: rgba(255, 255, 255, 0.95);
        padding: 3rem;
        border-radius: 20px;
        box-shadow: 0 20px 60px rgba(0, 0, 0, 0.3);
        max-width: 500px;
        margin: 2rem auto;
    }

    /* Title styling */
    .app-title {
        text-align: center;
        color: #ffffff;
        font-size: 3rem;
        font-weight: 800;
        margin-bottom: 0.5rem;
        text-shadow: 2px 2px 8px rgba(0,0,0,0.3);
    }

    .app-subtitle {
        text-align: center;
        color: #ffffff;
        font-size: 1.2rem;
        margin-bottom: 2rem;
        font-weight: 500;
        text-shadow: 1px 1px 4px rgba(0,0,0,0.2);
    }

    /* Button styling */
    .stButton > button {
        width: 100%;
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: white;
        border: none;
        padding: 0.75rem 2rem;
        border-radius: 50px;
        font-size: 1rem;
        font-weight: 600;
        transition: all 0.3s ease;
        box-shadow: 0 4px 15px rgba(102, 126, 234, 0.4);
    }

    .stButton > button:hover {
        transform: translateY(-2px);
        box-shadow: 0 6px 20px rgba(102, 126, 234, 0.6);
    }

    /* Input field styling - CRITICAL FIX FOR VISIBILITY */
    .stTextInput > div > div > input,
    .stSelectbox > div > div > select {
        border: 2px solid #e9ecef !important;
        border-radius: 10px !important;
        padding: 0.75rem !important;
        font-size: 1rem !important;
        transition: all 0.3s ease !important;
        background-color: #ffffff !important;
        color: #1a202c !important;
    }

    /* Force dark text in all input types */
    input[type="text"],
    input[type="email"],
    input[type="password"],
    input {
        color: #1a202c !important;
        background-color: #ffffff !important;
    }

    /* Input focus state */
    .stTextInput > div > div > input:focus,
    .stSelectbox > div > div > select:focus {
        border-color: #667eea !important;
        box-shadow: 0 0 0 3px rgba(102, 126, 234, 0.1) !important;
        color: #1a202c !important;
    }

    /* Label styling - FIXED FOR VISIBILITY */
    .stTextInput > label,
    .stSelectbox > label {
        color: #2d3748 !important;
        font-weight: 600 !important;
        font-size: 0.95rem !important;
        margin-bottom: 0.5rem !important;
    }

    /* Placeholder text styling */
    .stTextInput input::placeholder,
    input::placeholder {
        color: #718096 !important;
        opacity: 0.8 !important;
    }

    /* Ensure typed text is always dark and visible */
    .stTextInput input,
    .stTextInput div[data-baseweb="base-input"] input {
        color: #1a202c !important;
        -webkit-text-fill-color: #1a202c !important;
    }

    /* Success/Error message styling */
    .stSuccess {
        background-color: #d4edda;
        border-left: 4px solid #28a745;
        padding: 1rem;
        border-radius: 8px;
    }

    .stError {
        background-color: #f8d7da;
        border-left: 4px solid #dc3545;
        padding: 1rem;
        border-radius: 8px;
    }

    /* Info message styling */
    .stInfo {
        background-color: #d1ecf1;
        border-left: 4px solid #0dcaf0;
        padding: 1rem;
        border-radius: 8px;
    }

    /* Welcome card styling */
    .welcome-card {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: white;
        padding: 2rem;
        border-radius: 15px;
        text-align: center;
        margin-bottom: 2rem;
        box-shadow: 0 10px 30px rgba(0, 0, 0, 0.2);
    }

    .welcome-card h2 {
        margin: 0;
        font-size: 2rem;
    }

    .welcome-card p {
        margin: 0.5rem 0 0 0;
        font-size: 1.1rem;
        opacity: 0.95;
    }

    /* Page header styling */
    .page-header {
        text-align: center;
        color: #667eea;
        font-size: 2rem;
        font-weight: 700;
        margin-bottom: 2rem;
        padding-bottom: 1rem;
        border-bottom: 3px solid #e9ecef;
    }

    /* Hide Streamlit branding */
    #MainMenu {visibility: hidden;}
    footer {visibility: hidden;}
    header {visibility: hidden;}

    /* Divider styling */
    hr {
        margin: 2rem 0;
        border: none;
        height: 2px;
        background: linear-gradient(to right, transparent, #667eea, transparent);
    }

    /* Select dropdown styling */
    .stSelectbox select {
        color: #1a202c !important;
        background-color: #ffffff !important;
    }

    /* Additional text visibility fixes */
    div[data-baseweb="select"] > div {
        background-color: #ffffff !important;
        color: #1a202c !important;
    }
    </style>
    """, unsafe_allow_html=True)

# ================= DATABASE =================
def init_db():
    conn = sqlite3.connect("users.db", check_same_thread=False)
    c = conn.cursor()
    c.execute("""
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL,
    password BLOB NOT NULL,
    security_q TEXT NOT NULL,
    security_a BLOB NOT NULL
)
""")
    conn.commit()
    conn.close()

# ================= VALIDATION =================
def validate_email(email):
    return re.match(r"[^@]+@[^@]+\.[a-zA-Z]{2,}", email)

def validate_password(password):
    return password.isalnum() and len(password) >= 8

# ================= HASHING =================
def hash_text(text):
    return bcrypt.hashpw(text.encode(), bcrypt.gensalt())

def verify_text(text, hashed):
    return bcrypt.checkpw(text.encode(), hashed)

# ================= JWT =================
def create_token(email, username):
    payload = {
        "email": email,
        "username": username,
        "exp": datetime.datetime.utcnow() + datetime.timedelta(minutes=30)
    }
    return jwt.encode(payload, SECRET_KEY, algorithm="HS256")

def verify_token(token):
    try:
        return jwt.decode(token, SECRET_KEY, algorithms=["HS256"])
    except:
        return None

# ================= SESSION =================
if "page" not in st.session_state:
    st.session_state.page = "home"

if "reset_email" not in st.session_state:
    st.session_state.reset_email = None

if "question_shown" not in st.session_state:
    st.session_state.question_shown = False

if "answer_verified" not in st.session_state:
    st.session_state.answer_verified = False

# ================= MAIN =================
def main():
    st.set_page_config(
        page_title="PolicyNav - Secure Authentication",
        page_icon="🔐",
        layout="centered"
    )

    load_css()
    init_db()

    # ================= HOME =================
    if st.session_state.page == "home":
        st.markdown('<h1 class="app-title">🔐 PolicyNav</h1>', unsafe_allow_html=True)
        st.markdown('<p class="app-subtitle">Secure User Authentication System</p>', unsafe_allow_html=True)

        col1, col2, col3 = st.columns([1, 2, 1])

        with col2:
            st.markdown("<br>", unsafe_allow_html=True)

            st.markdown('<div class="welcome-card">'
                       '<h2>Welcome! 👋</h2>'
                       '<p>Navigate policies with confidence</p>'
                       '</div>', unsafe_allow_html=True)

            if st.button("🚀 Login", key="home_login"):
                st.session_state.page = "login"
                st.rerun()

            if st.button("📝 Sign Up", key="home_signup"):
                st.session_state.page = "signup"
                st.rerun()

            if st.button("🔑 Forgot Password", key="home_forgot"):
                st.session_state.page = "forgot"
                st.rerun()

    # ================= LOGIN =================
    elif st.session_state.page == "login":
        col1, col2, col3 = st.columns([1, 2, 1])

        with col2:
            st.markdown('<h2 class="page-header">🔐 Login</h2>', unsafe_allow_html=True)

            email = st.text_input("📧 Email Address", placeholder="your.email@example.com", key="login_email")
            password = st.text_input("🔒 Password", type="password", placeholder="Enter your password", key="login_password")

            st.markdown("<br>", unsafe_allow_html=True)

            if st.button("Login", key="login_btn"):
                if not email or not password:
                    st.error("⚠️ Please fill in all fields")
                else:
                    conn = sqlite3.connect("users.db")
                    c = conn.cursor()
                    c.execute("SELECT username, password FROM users WHERE email=?", (email,))
                    user = c.fetchone()
                    conn.close()

                    if user and verify_text(password, user[1]):
                        token = create_token(email, user[0])
                        st.session_state.token = token
                        st.session_state.username = user[0]
                        st.session_state.page = "dashboard"
                        st.rerun()
                    else:
                        st.error("❌ Invalid email or password")

            if st.button("← Back to Home", key="login_back"):
                st.session_state.page = "home"
                st.rerun()

    # ================= SIGNUP =================
    elif st.session_state.page == "signup":
        col1, col2, col3 = st.columns([1, 2, 1])

        with col2:
            st.markdown('<h2 class="page-header">📝 Create Account</h2>', unsafe_allow_html=True)

            username = st.text_input("👤 Username", placeholder="Choose a username", key="signup_username")
            email = st.text_input("📧 Email Address", placeholder="your.email@example.com", key="signup_email")
            password = st.text_input("🔒 Password", type="password", placeholder="Min 8 characters, alphanumeric", key="signup_password")
            confirm = st.text_input("🔒 Confirm Password", type="password", placeholder="Re-enter password", key="signup_confirm")

            security_q = st.selectbox("🔐 Security Question", [
                "What is your pet name?",
                "What is your favorite color?",
                "What is your favorite book?",
                "What is your favorite movie?",
                "Who is your favorite teacher?"
            ], key="signup_security_q")

            security_a = st.text_input("💬 Security Answer", placeholder="Your answer (no spaces)", key="signup_security_a")

            st.markdown("<br>", unsafe_allow_html=True)

            if st.button("Create Account", key="signup_btn"):
                if not (username and email and password and confirm and security_a):
                    st.error("⚠️ All fields are mandatory")
                elif not validate_email(email):
                    st.error("❌ Invalid email format")
                elif not validate_password(password):
                    st.error("❌ Password must be alphanumeric & 8+ characters")
                elif password != confirm:
                    st.error("❌ Passwords do not match")
                elif " " in password:
                    st.error("❌ Password cannot contain spaces")
                elif " " in security_a:
                    st.error("❌ Security answer cannot contain spaces")
                else:
                    conn = sqlite3.connect("users.db")
                    c = conn.cursor()
                    try:
                        c.execute("""
INSERT INTO users (username, email, password, security_q, security_a)
VALUES (?, ?, ?, ?, ?)
""", (
                            username,
                            email,
                            hash_text(password),
                            security_q,
                            hash_text(security_a.lower().strip())
                        ))
                        conn.commit()
                        st.success("✅ Account created successfully! Please login.")
                    except sqlite3.IntegrityError:
                        st.error("❌ Email already exists.")
                    conn.close()

            if st.button("← Back to Home", key="signup_back"):
                st.session_state.page = "home"
                st.rerun()

    # ================= FORGOT PASSWORD =================
    elif st.session_state.page == "forgot":
        col1, col2, col3 = st.columns([1, 2, 1])

        with col2:
            st.markdown('<h2 class="page-header">🔑 Reset Password</h2>', unsafe_allow_html=True)

            email = st.text_input("📧 Registered Email", placeholder="your.email@example.com", key="forgot_email")

            if st.button("Check Email", key="check_email"):
                if not email:
                    st.error("⚠️ Please enter your email")
                else:
                    conn = sqlite3.connect("users.db")
                    c = conn.cursor()
                    c.execute("SELECT security_q FROM users WHERE email=?", (email,))
                    user = c.fetchone()
                    conn.close()

                    if user:
                        st.session_state.reset_email = email
                        st.session_state.question_shown = True
                        st.success(f"✅ Security Question: **{user[0]}**")
                    else:
                        st.error("❌ Email not found in our records")

            if st.session_state.question_shown:
                st.markdown("<br>", unsafe_allow_html=True)
                answer = st.text_input("💬 Your Answer", placeholder="Enter your security answer", key="forgot_answer")

                if st.button("Verify Answer", key="verify_answer"):
                    if " " in answer:
                        st.error("❌ Security answer cannot contain spaces")
                    else:
                        conn = sqlite3.connect("users.db")
                        c = conn.cursor()
                        c.execute("SELECT security_a FROM users WHERE email=?", (st.session_state.reset_email,))
                        stored = c.fetchone()
                        conn.close()

                        if stored and verify_text(answer.lower().strip(), stored[0]):
                            st.session_state.answer_verified = True
                            st.success("✅ Answer verified! You can now reset your password.")
                        else:
                            st.error("❌ Incorrect security answer.")

            if st.session_state.answer_verified:
                st.markdown("<br>", unsafe_allow_html=True)
                new_pass = st.text_input("🔒 New Password", type="password", placeholder="Min 8 characters, alphanumeric", key="forgot_new_pass")
                confirm_pass = st.text_input("🔒 Confirm New Password", type="password", placeholder="Re-enter new password", key="forgot_confirm_pass")

                if st.button("Reset Password", key="reset_password"):
                    if new_pass != confirm_pass:
                        st.error("❌ Passwords do not match.")
                    elif not validate_password(new_pass):
                        st.error("❌ Password must be alphanumeric & 8+ characters")
                    elif " " in new_pass:
                        st.error("❌ Password cannot contain spaces")
                    else:
                        conn = sqlite3.connect("users.db")
                        c = conn.cursor()
                        c.execute("UPDATE users SET password=? WHERE email=?",
                                  (hash_text(new_pass), st.session_state.reset_email))
                        conn.commit()
                        conn.close()

                        st.success("✅ Password reset successful! Please login with your new password.")

                        st.session_state.question_shown = False
                        st.session_state.answer_verified = False
                        st.session_state.reset_email = None

            if st.button("← Back to Home", key="forgot_back"):
                st.session_state.page = "home"
                st.session_state.question_shown = False
                st.session_state.answer_verified = False
                st.rerun()

    # ================= DASHBOARD =================
    elif st.session_state.page == "dashboard":
        payload = verify_token(st.session_state.get("token"))

        if payload:
            col1, col2, col3 = st.columns([1, 2, 1])

            with col2:
                st.markdown(f'''
                <div class="welcome-card">
                    <h2>Welcome Back! 🎉</h2>
                    <p style="font-size: 1.5rem; margin-top: 1rem;">
                        <strong>{payload['username']}</strong>
                    </p>
                    <p style="opacity: 0.95; margin-top: 0.5rem;">
                        📧 {payload['email']}
                    </p>
                </div>
                ''', unsafe_allow_html=True)

                st.markdown("### 🎯 Quick Actions")
                st.info("🚀 Your dashboard is ready! Future features will appear here.")

                st.markdown("<br>", unsafe_allow_html=True)

                if st.button("🚪 Logout", key="logout_btn"):
                    st.session_state.clear()
                    st.session_state.page = "home"
                    st.rerun()
        else:
            st.session_state.page = "login"
            st.rerun()

if __name__ == "__main__":
    main()

Writing app.py


In [3]:
from pyngrok import ngrok
import subprocess
import time
import os

# 🔐 Your ngrok token
NGROK_AUTH_TOKEN = "39WpdQdXP2S3d9BCMRWK2srpidW_28LuFQaAGw4Q4xJVotaF5"

ngrok.set_auth_token(NGROK_AUTH_TOKEN)

print("🧹 Cleaning up...")

# Kill everything properly
os.system("pkill -9 streamlit")
os.system("pkill -9 ngrok")

# Disconnect all tunnels
try:
    tunnels = ngrok.get_tunnels()
    for tunnel in tunnels:
        ngrok.disconnect(tunnel.public_url)
        print(f"✅ Disconnected: {tunnel.public_url}")
except:
    pass

ngrok.kill()
time.sleep(5)  # Wait longer for cleanup

print("🚀 Starting fresh...")

# Start Streamlit
process = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", "8501", "--server.headless", "true"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

time.sleep(8)

# Create tunnel
public_url = ngrok.connect(8501)
print("\n" + "="*60)
print("✨ PolicyNav App is Live!")
print("="*60)
print(f"🌍 Public URL: {public_url}")
print("="*60)

🧹 Cleaning up...
🚀 Starting fresh...

✨ PolicyNav App is Live!
🌍 Public URL: NgrokTunnel: "https://chubby-vania-unwonderfully.ngrok-free.dev" -> "http://localhost:8501"


In [4]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('users.db')
df = pd.read_sql_query("SELECT id, username, email, security_q FROM users", conn)
conn.close()
print(df)

Empty DataFrame
Columns: [id, username, email, security_q]
Index: []
